In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import seaborn as sns
from datetime import datetime,timedelta
import re
from geopy.geocoders import Nominatim
from geopy import distance
import holidays

In [18]:
complete_orders=pd.read_csv("../data/cleaned_complete_orders.csv")
delivery_requests=pd.read_csv("../data/driver_locations_during_request.csv")

delivery request information


In [19]:
delivery_requests.shape

(1557740, 8)

In [20]:
delivery_requests.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1557740 entries, 0 to 1557739
Data columns (total 8 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   id             1557740 non-null  int64  
 1   order_id       1557740 non-null  int64  
 2   driver_id      1557740 non-null  int64  
 3   driver_action  1557740 non-null  object 
 4   lat            1557740 non-null  float64
 5   lng            1557740 non-null  float64
 6   created_at     0 non-null        float64
 7   updated_at     0 non-null        float64
dtypes: float64(4), int64(3), object(1)
memory usage: 95.1+ MB


since we have no data for the `created_at` and `updated_at`, we can drop the columns as a whole

In [21]:
delivery_requests.drop(columns=["created_at","updated_at"],inplace=True)
delivery_requests.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1557740 entries, 0 to 1557739
Data columns (total 6 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   id             1557740 non-null  int64  
 1   order_id       1557740 non-null  int64  
 2   driver_id      1557740 non-null  int64  
 3   driver_action  1557740 non-null  object 
 4   lat            1557740 non-null  float64
 5   lng            1557740 non-null  float64
dtypes: float64(2), int64(3), object(1)
memory usage: 71.3+ MB


In [22]:
# there is no missing values in our dataset
delivery_requests.isna().sum()

id               0
order_id         0
driver_id        0
driver_action    0
lat              0
lng              0
dtype: int64

### Merge necessary records

In [23]:
complete_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536020 entries, 0 to 536019
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   trip_id           536020 non-null  int64  
 1   trip_origin       536020 non-null  object 
 2   trip_destination  536020 non-null  object 
 3   trip_start_time   536020 non-null  object 
 4   trip_end_time     536020 non-null  object 
 5   duration_min      536020 non-null  float64
 6   distance_km       536020 non-null  float64
 7   holiday           536020 non-null  bool   
dtypes: bool(1), float64(2), int64(1), object(4)
memory usage: 29.1+ MB


### Merge assemptions

- we are going to merge the two datasets on `order_id` eqauls `trip_id`
- we already have the origin location, so we are going to take only the destination location
- only the `trip start time` has effect on drivers action, so we are going to take only the trip start time
- we take distance in km
- we take holiday status


In [26]:
useful_info = complete_orders[["trip_id","trip_destination","trip_start_time","distance_km","holiday"]]

In [33]:
lat = useful_info["trip_destination"].apply(lambda x:x.split(',')[0])
lon = useful_info["trip_destination"].apply(lambda x:x.split(',')[1])
useful_info['dest_lat'],useful_info['dest_lng']=lat,lon
useful_info.drop(columns="trip_destination",inplace=True)

C:\Users\Natnael\AppData\Local\Temp\ipykernel_11916\2041192716.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useful_info['dest_lat'],useful_info['dest_lng']=lat,lon
C:\Users\Natnael\AppData\Local\Temp\ipykernel_11916\2041192716.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useful_info.drop(columns="trip_destination",inplace=True)


In [13]:
delivery_requests.rename(columns={"lat":"origin_lat","lng":"origin_lng"},inplace=True)

In [14]:
delivery_requests.head()

,id,order_id,driver_id,driver_action,origin_lat,origin_lng
0,1,392001,243828,accepted,6.602207,3.270465
1,2,392001,243588,rejected,6.592097,3.287445
2,3,392001,243830,rejected,6.596133,3.281784
3,4,392001,243539,rejected,6.596142,3.280526
4,5,392001,171653,rejected,6.609232,3.288800


In [11]:
complete_orders[complete_orders["trip_id"]==392001]


,trip_id,trip_origin,trip_destination,trip_start_time,trip_end_time,duration_min,distance_km,holiday
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,3.0,20.984319,False


In [ ]:
delivery_requests["driver_action"].value_counts()
